In [1]:
%autosave 1

Autosaving every 1 seconds


In [2]:
import numpy as np
import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import os
from torch.utils.data import DataLoader, dataloader
from torch import nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import cv2
import torch.nn.functional as func
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
! nvidia-smi

cuda
Mon Aug 15 09:17:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
|  0%   40C    P8    13W / 130W |     65MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [4]:
transform=transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                             ])

In [5]:
root_dir=os.path.join(os.getcwd(),'cifar')
train_dataset = datasets.ImageFolder(os.path.join(root_dir,'train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(root_dir,'test'), transform=transform)
train_set, val_set = torch.utils.data.random_split(train_dataset, [25000, 5000])
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
len(train_dataset)

30000

In [6]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [7]:
class Cifar_Lin(nn.Module):
    def __init__(self):
        super(Cifar_Lin,self).__init__()
        self.layer1 = nn.Linear(3072,768)
        self.layer2 = nn.Linear(768,192)
        self.Final = nn.Linear(192,6)
        self.relu = nn.ReLU()
    def forward(self, data):
        mtx = torch.flatten(data, start_dim=1)
        mtx = self.layer1(mtx)
        mtx = self.relu(mtx)
        mtx = self.layer2(mtx)
        mtx = self.relu(mtx)
        mtx = self.Final(mtx)
        return mtx

In [8]:
criterion = nn.CrossEntropyLoss()
model = Cifar_Lin().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
epochs = 300
time0 = time()

for e_no in range(1, epochs+1):
    running_loss = 0
    batch_no = 0
    correct = 0 
    total = 0
    for images, labels in train_loader:
      if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())
      batch_no += 1
      model.train()
      images = images.flatten(start_dim=1)
      optimizer.zero_grad()
      output = model(images)
      _, output_index = output.topk(1)
      if torch.cuda.is_available():
        result = labels.cpu().numpy() == output_index.cpu().numpy().reshape(-1,)
        #print("executing in cuda")
      else:
        result = labels.numpy() == output_index.numpy().reshape(-1,)
      correct += result.sum()
      total += len(labels)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    else:
      print("Epoch {} - Training Loss: {}".format(e_no, running_loss/len(train_loader)))
      print(f"Training Accuracy: {(correct/total) * 100}")
      print("Training Time in Minute=",(time()-time0)/60)
      model.eval()
      correct = 0
      total = 0
      for val_images, vlabels in test_loader:
        if torch.cuda.is_available():
          val_images = Variable(val_images.cuda())
          vlabels = Variable(vlabels.cuda())
        val_images = val_images.flatten(start_dim=1)
        val_output = model(val_images)
        _, output_idx = val_output.topk(1)
        if torch.cuda.is_available():
          res = vlabels.cpu().numpy() == output_idx.cpu().numpy().reshape(-1,)
        else:
          res = vlabels.numpy() == output_idx.numpy().reshape(-1,)
        correct += res.sum()
        total += len(vlabels)
            
      print(f"Validation Accuracy: {(correct * 100) / total}\n")

Epoch 1 - Training Loss: 1.5980450360061567
Training Accuracy: 38.852
Training Time in Minute= 4.4015688379605615
Validation Accuracy: 44.88333333333333

Epoch 2 - Training Loss: 1.3864170035437855
Training Accuracy: 46.656
Training Time in Minute= 4.845875418186187
Validation Accuracy: 48.45

Epoch 3 - Training Loss: 1.3076557000274853
Training Accuracy: 49.628
Training Time in Minute= 5.173211908340454
Validation Accuracy: 49.8

Epoch 4 - Training Loss: 1.26420180236592
Training Accuracy: 51.160000000000004
Training Time in Minute= 5.500560049215952
Validation Accuracy: 51.21666666666667

Epoch 5 - Training Loss: 1.2333766568042432
Training Accuracy: 52.532000000000004
Training Time in Minute= 5.829278322060903
Validation Accuracy: 52.61666666666667

Epoch 6 - Training Loss: 1.2090230436276292
Training Accuracy: 53.388000000000005
Training Time in Minute= 6.156562968095144
Validation Accuracy: 53.2

Epoch 7 - Training Loss: 1.1882008800421224
Training Accuracy: 54.288000000000004
Tra

Validation Accuracy: 59.68333333333333

Epoch 55 - Training Loss: 0.714525913338527
Training Accuracy: 74.908
Training Time in Minute= 22.214026268323263
Validation Accuracy: 60.35

Epoch 56 - Training Loss: 0.7073766228640476
Training Accuracy: 75.28
Training Time in Minute= 22.541427799065907
Validation Accuracy: 60.016666666666666

Epoch 57 - Training Loss: 0.6987153774941973
Training Accuracy: 75.62
Training Time in Minute= 22.870653780301414
Validation Accuracy: 60.166666666666664

Epoch 58 - Training Loss: 0.6910534219058884
Training Accuracy: 76.024
Training Time in Minute= 23.196694513161976
Validation Accuracy: 60.46666666666667

Epoch 59 - Training Loss: 0.6833483852693797
Training Accuracy: 76.556
Training Time in Minute= 23.524526488780975
Validation Accuracy: 60.25

Epoch 60 - Training Loss: 0.6757079987879604
Training Accuracy: 76.596
Training Time in Minute= 23.853144363562265
Validation Accuracy: 60.5

Epoch 61 - Training Loss: 0.6674164201292541
Training Accuracy: 77.1

Validation Accuracy: 60.61666666666667

Epoch 109 - Training Loss: 0.3280909990182008
Training Accuracy: 91.708
Training Time in Minute= 39.846421309312184
Validation Accuracy: 60.15

Epoch 110 - Training Loss: 0.32143394206948295
Training Accuracy: 92.012
Training Time in Minute= 40.172387246290846
Validation Accuracy: 60.5

Epoch 111 - Training Loss: 0.31522738605813905
Training Accuracy: 92.16
Training Time in Minute= 40.498932643731436
Validation Accuracy: 60.43333333333333

Epoch 112 - Training Loss: 0.30935505920511375
Training Accuracy: 92.528
Training Time in Minute= 40.823267714182535
Validation Accuracy: 60.5

Epoch 113 - Training Loss: 0.3026535235859854
Training Accuracy: 92.51599999999999
Training Time in Minute= 41.14904630978902
Validation Accuracy: 60.21666666666667

Epoch 114 - Training Loss: 0.2966435708658165
Training Accuracy: 92.99199999999999
Training Time in Minute= 41.47518803278605
Validation Accuracy: 60.483333333333334

Epoch 115 - Training Loss: 0.2921324732

Validation Accuracy: 59.55

Epoch 162 - Training Loss: 0.0885185497381803
Training Accuracy: 99.26
Training Time in Minute= 57.11940765778224
Validation Accuracy: 59.63333333333333

Epoch 163 - Training Loss: 0.08545459210491546
Training Accuracy: 99.32
Training Time in Minute= 57.44405254125595
Validation Accuracy: 59.55

Epoch 164 - Training Loss: 0.0830306447566013
Training Accuracy: 99.36
Training Time in Minute= 57.77074892918269
Validation Accuracy: 59.86666666666667

Epoch 165 - Training Loss: 0.08030595948629062
Training Accuracy: 99.432
Training Time in Minute= 58.09587567249934
Validation Accuracy: 59.68333333333333

Epoch 166 - Training Loss: 0.07712159583063992
Training Accuracy: 99.488
Training Time in Minute= 58.42236349980036
Validation Accuracy: 59.78333333333333

Epoch 167 - Training Loss: 0.07565494445736146
Training Accuracy: 99.456
Training Time in Minute= 58.74747300942739
Validation Accuracy: 59.53333333333333

Epoch 168 - Training Loss: 0.07285284345297863
Traini

Epoch 215 - Training Loss: 0.014253024145236711
Training Accuracy: 99.94
Training Time in Minute= 74.40292778809865
Validation Accuracy: 59.95

Epoch 216 - Training Loss: 0.015508224945896498
Training Accuracy: 99.924
Training Time in Minute= 74.72839519580205
Validation Accuracy: 59.4

Epoch 217 - Training Loss: 0.011390543377260342
Training Accuracy: 99.984
Training Time in Minute= 75.05389379262924
Validation Accuracy: 59.75

Epoch 218 - Training Loss: 0.011581428121546727
Training Accuracy: 99.98
Training Time in Minute= 75.37945053180059
Validation Accuracy: 59.93333333333333

Epoch 219 - Training Loss: 0.01162595354625598
Training Accuracy: 99.97200000000001
Training Time in Minute= 75.70609138011932
Validation Accuracy: 59.483333333333334

Epoch 220 - Training Loss: 0.01240559608992332
Training Accuracy: 99.964
Training Time in Minute= 76.03169872363408
Validation Accuracy: 59.85

Epoch 221 - Training Loss: 0.011689983822805497
Training Accuracy: 99.964
Training Time in Minute= 

Epoch 269 - Training Loss: 0.0026764355503413302
Training Accuracy: 100.0
Training Time in Minute= 92.01610567569733
Validation Accuracy: 59.46666666666667

Epoch 270 - Training Loss: 0.014417854640001665
Training Accuracy: 99.668
Training Time in Minute= 92.34085823694865
Validation Accuracy: 58.083333333333336

Epoch 271 - Training Loss: 0.00694572729358683
Training Accuracy: 99.908
Training Time in Minute= 92.66588887373607
Validation Accuracy: 59.11666666666667

Epoch 272 - Training Loss: 0.002941748187186487
Training Accuracy: 100.0
Training Time in Minute= 92.9922404050827
Validation Accuracy: 59.3

Epoch 273 - Training Loss: 0.002665427145119423
Training Accuracy: 100.0
Training Time in Minute= 93.3171801606814
Validation Accuracy: 59.36666666666667

Epoch 274 - Training Loss: 0.002502657943214659
Training Accuracy: 100.0
Training Time in Minute= 93.64347701867422
Validation Accuracy: 59.3

Epoch 275 - Training Loss: 0.0024059801186789827
Training Accuracy: 100.0
Training Time i

In [9]:
class Cifer_CNN(nn.Module):
    def __init__(self):
        super(Cifer_CNN,self).__init__()
        layer1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, padding=1, stride=1)
        layer2=nn.Conv2d(in_channels=12, out_channels=32, kernel_size=3, padding=1, stride=1)
        layer3=nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1)
        layer4=nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1)
        layer5=nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1)
        layer6=nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1)
        linear=nn.Linear(131072,1024)
        final=nn.Linear(1024,6)
        maxpool=nn.MaxPool2d(kernel_size=2)
        relu=nn.ReLU()
        batch_norm1=nn.BatchNorm2d(12)
        batch_norm2=nn.BatchNorm2d(32)
        batch_norm3=nn.BatchNorm2d(64)
        batch_norm4=nn.BatchNorm2d(128)
        batch_norm5=nn.BatchNorm2d(256)
        batch_norm6=nn.BatchNorm2d(512)
    def forward(self, data):
        mtx1 = self.relu(func.batchnorm1(self.layer1(data)))
        mtx1 = maxpool(mtx1)
        mtx1 = self.relu(func.batchnorm2(self.layer2(mtx1)))
        mtx1 = self.relu(func.batchnorm3(self.layer3(mtx1)))
        mtx1 = self.relu(func.batchnorm4(self.layer4(mtx1)))
        mtx1 = self.relu(func.batchnorm5(self.layer5(mtx1)))
        mtx1 = self.relu(func.batchnorm6(self.layer6(mtx1)))
        mtx1 = torch.flatten(mtx1, start_dim=1)
        mtx1 = func.relu(self.linear(mtx1))
        mtx1 = self.Final(mtx1)
        return mtx1

In [10]:
criterion1 = nn.CrossEntropyLoss()
model1 = Cifer_CNN().to(device)
optimizer1 = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.0001)
epochs = 500
time0 = time()

for e_no in range(1, epochs+1):
    running_loss = 0
    batch_no = 0
    correct = 0 
    total = 0
    for images, labels in train_loader:
      if torch.cuda.is_available():
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())
      batch_no += 1
      model1.train()
      #images = images.flatten(start_dim=1)
      optimizer1.zero_grad()
      output = model(images)
      _, output_index = output.topk(1)
      if torch.cuda.is_available():
        result = labels.cpu().numpy() == output_index.cpu().numpy().reshape(-1,)
      else:
        result = labels.numpy() == output_index.numpy().reshape(-1,)
      correct += result.sum()
      total += len(labels)
      loss = criterion1(output, labels)
      loss.backward()
      optimizer1.step()
      running_loss += loss.item()
    else:
        print("Epoch {} - Training Loss: {}".format(e_no, running_loss/len(train_loader)))
        print(f"Training Accuracy: {(correct/total) * 100}")
        print("Training Time in Minute=",(time()-time0)/60)
        model1.eval()
        correct = 0
        total = 0
        for val_images, vlabels in test_loader:
          if torch.cuda.is_available():
            val_images = Variable(val_images.cuda())
            vlabels = Variable(vlabels.cuda())
          val_images = val_images.flatten(start_dim=1)
          val_output = model(val_images)
          _, output_idx = val_output.topk(1)
          res = vlabels.cpu().numpy() == output_idx.cpu().numpy().reshape(-1,)
          correct += res.sum()
          total += len(vlabels)
            
        print(f"Validation Accuracy: {(correct * 100) / total}\n")

Epoch 1 - Training Loss: 0.2885113954344583
Training Accuracy: 92.536
Training Time in Minute= 0.2898682475090027
Validation Accuracy: 57.3

Epoch 2 - Training Loss: 0.1601952861117966
Training Accuracy: 94.976
Training Time in Minute= 0.6219444155693055
Validation Accuracy: 55.5

Epoch 3 - Training Loss: 0.24051594268292417
Training Accuracy: 93.22
Training Time in Minute= 0.9544422507286072
Validation Accuracy: 55.93333333333333

Epoch 4 - Training Loss: 0.15047394782018936
Training Accuracy: 95.304
Training Time in Minute= 1.2860496878623962
Validation Accuracy: 55.61666666666667

Epoch 5 - Training Loss: 0.25539582956801443
Training Accuracy: 92.72399999999999
Training Time in Minute= 1.6194462736447652
Validation Accuracy: 56.55

Epoch 6 - Training Loss: 0.20019445230093452
Training Accuracy: 94.04
Training Time in Minute= 1.9519391576449077
Validation Accuracy: 56.28333333333333

Epoch 7 - Training Loss: 0.13584826034886757
Training Accuracy: 95.66799999999999
Training Time in Mi

Epoch 55 - Training Loss: 0.04383006574296633
Training Accuracy: 98.72
Training Time in Minute= 18.24067940711975
Validation Accuracy: 57.8

Epoch 56 - Training Loss: 0.04541513415903825
Training Accuracy: 98.676
Training Time in Minute= 18.57326509952545
Validation Accuracy: 56.1

Epoch 57 - Training Loss: 0.06311192214750992
Training Accuracy: 97.86
Training Time in Minute= 18.905246778329214
Validation Accuracy: 56.63333333333333

Epoch 58 - Training Loss: 0.09906267816596248
Training Accuracy: 96.832
Training Time in Minute= 19.23707687854767
Validation Accuracy: 57.86666666666667

Epoch 59 - Training Loss: 0.06884964583012873
Training Accuracy: 97.816
Training Time in Minute= 19.569918525218963
Validation Accuracy: 58.43333333333333

Epoch 60 - Training Loss: 0.023057162027765553
Training Accuracy: 99.42800000000001
Training Time in Minute= 19.909029734134673
Validation Accuracy: 58.15

Epoch 61 - Training Loss: 0.12022488140929824
Training Accuracy: 96.48
Training Time in Minute=

Validation Accuracy: 59.63333333333333

Epoch 109 - Training Loss: 0.0008886581049491282
Training Accuracy: 100.0
Training Time in Minute= 36.194674070676164
Validation Accuracy: 59.766666666666666

Epoch 110 - Training Loss: 0.23331276760389702
Training Accuracy: 94.664
Training Time in Minute= 36.52609876394272
Validation Accuracy: 58.46666666666667

Epoch 111 - Training Loss: 0.07386601460464012
Training Accuracy: 97.796
Training Time in Minute= 36.858283853530885
Validation Accuracy: 57.2

Epoch 112 - Training Loss: 0.025790661060110763
Training Accuracy: 99.312
Training Time in Minute= 37.18959069252014
Validation Accuracy: 59.3

Epoch 113 - Training Loss: 0.03182839639359833
Training Accuracy: 99.132
Training Time in Minute= 37.52246505022049
Validation Accuracy: 58.0

Epoch 114 - Training Loss: 0.08728752466028228
Training Accuracy: 97.184
Training Time in Minute= 37.85443435112635
Validation Accuracy: 57.71666666666667

Epoch 115 - Training Loss: 0.012670146878106076
Training A

Validation Accuracy: 59.36666666666667

Epoch 163 - Training Loss: 0.0007264864071027574
Training Accuracy: 100.0
Training Time in Minute= 54.13776530424754
Validation Accuracy: 59.483333333333334

Epoch 164 - Training Loss: 0.0006886389235432362
Training Accuracy: 100.0
Training Time in Minute= 54.46933253208796
Validation Accuracy: 59.45

Epoch 165 - Training Loss: 0.24914680879346937
Training Accuracy: 93.812
Training Time in Minute= 54.8018239180247
Validation Accuracy: 58.666666666666664

Epoch 166 - Training Loss: 0.04061031144331483
Training Accuracy: 98.856
Training Time in Minute= 55.133115990956625
Validation Accuracy: 58.46666666666667

Epoch 167 - Training Loss: 0.01465227067931448
Training Accuracy: 99.664
Training Time in Minute= 55.465830369790396
Validation Accuracy: 59.233333333333334

Epoch 168 - Training Loss: 0.0029193586532903068
Training Accuracy: 99.988
Training Time in Minute= 55.79908099571864
Validation Accuracy: 59.666666666666664

Epoch 169 - Training Loss: 

Validation Accuracy: 59.95

Epoch 216 - Training Loss: 0.0009398160190732382
Training Accuracy: 100.0
Training Time in Minute= 71.68361732165019
Validation Accuracy: 60.03333333333333

Epoch 217 - Training Loss: 0.0008127182750972202
Training Accuracy: 100.0
Training Time in Minute= 72.01448412736256
Validation Accuracy: 59.95

Epoch 218 - Training Loss: 0.0007542406486155575
Training Accuracy: 100.0
Training Time in Minute= 72.34569110472997
Validation Accuracy: 59.9

Epoch 219 - Training Loss: 0.0007282304851075663
Training Accuracy: 100.0
Training Time in Minute= 72.67630026340484
Validation Accuracy: 59.766666666666666

Epoch 220 - Training Loss: 0.0006755958855002666
Training Accuracy: 100.0
Training Time in Minute= 73.00774784088135
Validation Accuracy: 59.983333333333334

Epoch 221 - Training Loss: 0.0006612545311631983
Training Accuracy: 100.0
Training Time in Minute= 73.33961096604665
Validation Accuracy: 59.75

Epoch 222 - Training Loss: 0.0006495638224019376
Training Accurac

Validation Accuracy: 60.45

Epoch 270 - Training Loss: 0.0008181956530202661
Training Accuracy: 100.0
Training Time in Minute= 89.6252356449763
Validation Accuracy: 60.266666666666666

Epoch 271 - Training Loss: 0.15212158038033186
Training Accuracy: 95.44399999999999
Training Time in Minute= 89.95811590353648
Validation Accuracy: 56.28333333333333

Epoch 272 - Training Loss: 0.0774671026384053
Training Accuracy: 97.548
Training Time in Minute= 90.28950779835382
Validation Accuracy: 58.61666666666667

Epoch 273 - Training Loss: 0.02836189494150288
Training Accuracy: 99.264
Training Time in Minute= 90.62276457945505
Validation Accuracy: 59.06666666666667

Epoch 274 - Training Loss: 0.007243325172499527
Training Accuracy: 99.912
Training Time in Minute= 90.95485844612122
Validation Accuracy: 58.81666666666667

Epoch 275 - Training Loss: 0.002960777899418312
Training Accuracy: 99.988
Training Time in Minute= 91.28555679321289
Validation Accuracy: 59.7

Epoch 276 - Training Loss: 0.0819432

Validation Accuracy: 58.68333333333333

Epoch 324 - Training Loss: 0.009658386497764284
Training Accuracy: 99.78399999999999
Training Time in Minute= 107.56642763614654
Validation Accuracy: 59.18333333333333

Epoch 325 - Training Loss: 0.03538101433170126
Training Accuracy: 98.98400000000001
Training Time in Minute= 107.89963837067286
Validation Accuracy: 58.63333333333333

Epoch 326 - Training Loss: 0.009486970449875221
Training Accuracy: 99.79599999999999
Training Time in Minute= 108.2322119474411
Validation Accuracy: 59.46666666666667

Epoch 327 - Training Loss: 0.0022772665177543394
Training Accuracy: 99.98
Training Time in Minute= 108.56318816343943
Validation Accuracy: 59.56666666666667

Epoch 328 - Training Loss: 0.0009127182345908812
Training Accuracy: 100.0
Training Time in Minute= 108.89629288514455
Validation Accuracy: 59.61666666666667

Epoch 329 - Training Loss: 0.0007499434439318917
Training Accuracy: 100.0
Training Time in Minute= 109.2282009959221
Validation Accuracy: 5

Epoch 377 - Training Loss: 0.1125004457004959
Training Accuracy: 96.17999999999999
Training Time in Minute= 125.28227827151616
Validation Accuracy: 59.1

Epoch 378 - Training Loss: 0.020778489892806888
Training Accuracy: 99.464
Training Time in Minute= 125.61548399527868
Validation Accuracy: 58.766666666666666

Epoch 379 - Training Loss: 0.017358141351322097
Training Accuracy: 99.536
Training Time in Minute= 125.9508907755216
Validation Accuracy: 58.5

Epoch 380 - Training Loss: 0.018479605348271026
Training Accuracy: 99.512
Training Time in Minute= 126.28625393708548
Validation Accuracy: 57.666666666666664

Epoch 381 - Training Loss: 0.040334225703111806
Training Accuracy: 98.80799999999999
Training Time in Minute= 126.61953329642614
Validation Accuracy: 57.56666666666667

Epoch 382 - Training Loss: 0.034569055131395986
Training Accuracy: 98.952
Training Time in Minute= 126.95189781983693
Validation Accuracy: 59.35

Epoch 383 - Training Loss: 0.002234031302470933
Training Accuracy: 10

KeyboardInterrupt: 

In [ ]:
def get_dataset(dataset_dir): 
    train_x = [] 
    train_y = []
    directories=os.listdir(dataset_dir)
    for dir in directories:
      for file in os.scandir(os.path.join(dataset_dir,dir)):
        img_path=os.path.join(os.path.join(dataset_dir,dir),file)
        img=cv2.imread(img_path)
        img=np.array(img)
        train_x.append(img.flatten())
        train_y.append(dir)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    return train_x, train_y

In [ ]:
X_train, y_train = get_dataset(os.path.join(root_dir,'train'))
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_val, y_val = get_dataset(os.path.join(root_dir,'test'))
print(X_val.shape)
print(y_val.shape)

In [ ]:
pca = PCA(n_components = 100)
pca_train=pca.fit_transform(X_train)
x_train=pca_train
pca_val=pca.fit_transform(X_val)
x_val=pca_val
x_train.shape

In [ ]:
poly_svc = SVC(kernel='rbf')
poly_svc.fit(x_train, y_train)
accuracy_poly = poly_svc.score(x_val, y_val)
print("Polynomial SVM Accuracy: ", accuracy_poly * 100, "%")